In [1]:
import pandas as pd
import os
from dataclasses import dataclass, field
import sys
sys.path.insert(0, '/home/jovyan/work')
sys.path.insert(1, '/home/jovyan/work/src')
sys.path.insert(2, '/home/jovyan/work/src/service')

from service import bfs_cases_db_service as bfs_db

from py.global_configs import *
from py.normalize import normalize

/home/jovyan/work/src/models/BfsCase.py:5: SAWarning: On class 'BfsCase', Column object 'drg_cost_weight' named directly multiple times, only one will be used: drg_cost_weight, effective_cost_weight. Consider using orm.synonym instead
  class BfsCase(Base):


In [2]:
# check all the file name

FILES_TO_ANALYZE.keys()


dict_keys(['Hirslanden Salem 2017', 'Hirslanden Beau Site 2017', 'Hirslanden Linde 2017', 'Hirslanden Linde 2018', 'Hirslanden Salem 2018', 'Hirslanden Beau Site 2018', 'Winterthur 2017', 'Winterthur 2018', 'Winterthur 2019', 'Winterthur 2020', 'USZ 2018', 'USZ 2019', 'USZ 2019_30_04_2020', 'KSSG 2021'])

In [3]:
file = FILES_TO_ANALYZE['USZ 2018']
file

FileInfo(path='/home/jovyan/work/src/revised_case_normalization/raw_data/USZ_2018-2019_20200730.xlsx', hospital_name_db='USZ', year='2018', sheets=['Rückmeldungen_USZ_2018'])

In [4]:

df_revised_case_d2d = normalize(file, 0)

Read 169 cases for USZ 2018
6/169 rows were deleted because contained NaNs
TYPES:
case_id             string
patient_id          string
gender              string
age_years            int64
duration_of_stay     int64
pccl                 int64
drg                 string
pd                  string
bfs_code            string
added_icds          string
removed_icds        string
added_chops         string
removed_chops       string
dtype: object


In [5]:
df_revised_case_d2d.head()

,case_id,patient_id,gender,age_years,duration_of_stay,pccl,drg,pd,bfs_code,added_icds,removed_icds,added_chops,removed_chops
0,5771703,4424387,W,90,10,3,L63D,N390,<NA>,U804,<NA>,"8688j3::20171228,8688j3::20171231","8688J3::20171228,8688J3::20171231"
1,5773574,60210855,M,95,14,3,J10Z,D042,<NA>,B965,<NA>,009b11::20171229,009B11::20171229
2,5776284,10743014,W,83,5,3,F12D,I495,<NA>,"I5014,J4409","I5012,J4483","378a41::20180103,39a211::20180103,39a212::2018...","378A41::20180103,39A211::20180103,39A212::2018..."
3,5793305,6214207,M,60,7,3,E71B,C343,<NA>,"J9600,J90",<NA>,"890a21::20180116,3491::20180117",890A21::20180116
4,5816220,10776936,W,84,8,3,F24C,I214,<NA>,N184,N183,<NA>,<NA>


# match to the database


In [6]:
# get the case_id from revised_case

revised_case_id = df_revised_case_d2d['case_id'].values
revised_case_id

array(['5771703', '5773574', '5776284', '5793305', '5816220', '5817015',
       '5818634', '5820123', '5835580', '5847678', '5918824', '6012516',
       '6015473', '5767666', '5833591', '5983561', '5988235', '5773539',
       '5791315', '5803681', '5816899', '5847689', '5855548', '5856159',
       '5858837', '5865426', '5870003', '5874948', '5810479', '5876676',
       '5880915', '5887960', '5888030', '5891624', '5893301', '5894533',
       '5936690', '5944999', '5976340', '5976741', '6000152', '6057861',
       '5940470', '5949447', '5956139', '5960301', '5962510', '5964523',
       '5974406', '5974555', '5982915', '5998712', '6006947', '6007651',
       '6015931', '6017464', '6022613', '6040114', '6042542', '5946235',
       '5979084', '5981888', '5989742', '6005651', '6022196', '6025626',
       '6046039', '6088668', '6089811', '6100313', '6124640', '6125460',
       '5831321', '5914238', '5981844', '6010373', '6038064', '6049732',
       '6088597', '6110160', '6119451', '6123005', 

In [7]:
# match to the database
revised_case_db = bfs_db.get_bfs_cases_by_ids(revised_case_id)
revised_case_db.head()

,drg_cost_weight,aimedic_id,hospital_id,case_id,patient_id,age_years,age_days,gender,duration_of_stay,clinic_id,ventilation_hours,admission_weight,gestation_age,admission_date,admission_type,discharge_date,discharge_type,drg,adrg,pccl


In [8]:
# 
print('There are {} out of {} revised cases from DtoD that are matched with the database for {} {}'.format(len(revised_case_db), len(revised_case_id), file.hospital_name_db, file.year))

There are 0 out of 163 revised cases from DtoD that are matched with the database for USZ 2018


In [9]:
# if we find the match cases, then we need to check if the case_id, gender, year....are match